In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import pickle

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import collections
import time
import pickle
import re

In [2]:
# hw_dir = '/Users/Brian/Desktop/MLDS/FinalProject/CNN'
# hw_dir = '/Users/yao/Desktop/trueman/big4/MLDS/final'
hw_dir = '/home/yao/gm/mlds/final'
test_max_slen = 20
train_softmax_slen = 50
train_max_slen = 70

max_slen = 43
min_slen = 7

batch_size = 1280

In [3]:
with open('%s/all_pickles'%hw_dir,'rb') as f1:
    [train_data,test_data,word2ix,ix2word,vocab_size] = pickle.load(f1)

In [4]:

blank_ix = word2ix[' ']
new_data = []
for data in train_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
    new_data.append(temp)
train_data = new_data
train_data = [i for i in train_data if len(i[1])<=70 and len(i[1])>=5]

blank_ix = word2ix[' ']
new_data = []
for data in test_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
#     temp[1].extend([13]*(max_slen-len(temp[1])))
    new_data.append(temp)
test_data = new_data

In [5]:
amax = [len(i[1]) for i in test_data]

In [6]:
min(amax)

7

In [7]:
len(train_data),len(test_data)

(101930, 9264)

In [8]:
train_data_dict = dict()
for i in train_data:
    len_cmmt = len(i[1])
    if len_cmmt in train_data_dict:
        train_data_dict[len_cmmt].append(i)
    else:
        train_data_dict[len_cmmt] = [i]

In [9]:
train_data_dict = dict()
for i in test_data:
    len_cmmt = len(i[1])
    if len_cmmt in train_data_dict:
        train_data_dict[len_cmmt].append(i)
    else:
        train_data_dict[len_cmmt] = [i]

In [10]:
# len_pos    = max_slen  # 現在在長度多少的file，從長的開始
# batch_pos  = 0
# def get_batch(batch_size):
#     global train_data_dict, len_pos, batch_pos
#     batch_size = (batch_size//len_pos)+1 # 越長的句子一個batch就讀越少句，+1是避免0的狀況
#     # ====================回傳用的參數=============================
#     is_eoi = False  # end of iteration
#     is_eof = False
#     slen  = len_pos
#     # ===========================================================
#     # ====================從檔案中讀batch_size句===================
#     now_datas = train_data_dict[len_pos][batch_pos:batch_pos+batch_size]
#     batch_pos+=batch_size
#     if len(now_datas)<batch_size:
#         batch_pos=0
#         is_eof = True
#     # ===========================================================
#     batch_size = len(now_datas) # 因為有可能在檔案的尾巴 讀不滿完整的batch_size 故在此更新batch_size
#     # ==================若現在的檔案讀完============================
#     if is_eof == True:
#         random.shuffle(train_data_dict[len_pos])
#         if len_pos == min_slen: # 一個iteration跑完 重新再跑
#             len_pos = max_slen
#             is_eoi = True
#         else:                      # 現在這長度的file完了 開始讀len_pos-1的檔案
#             len_pos -= 1
#         if batch_size == 0: # 剛好讀到尾巴了
#             return is_eoi,is_eof,None,None,slen,batch_size
#     # ============================================================
        
#     score = np.array([i[0] for i in now_datas])
#     cmmt = np.array([i[1] for i in now_datas])
#     return is_eoi,is_eof,score,cmmt,slen,batch_size

In [11]:
min_slen


7

In [12]:
len_pos    = max_slen  # 現在在長度多少的file，從長的開始
pad_ix = word2ix['<PAD>']
batch_pos  = 0
def get_batch(batch_size):
    global train_data_dict, len_pos, batch_pos
    batch_size = (batch_size//len_pos)+1 # 越長的句子一個batch就讀越少句，+1是避免0的狀況
    # ====================回傳用的參數=============================
    is_eoi = False  # end of iteration
    is_eof = False
    slen  = len_pos
    # ===========================================================
    # ====================從檔案中讀batch_size句===================
#     try:
    now_datas = train_data_dict[len_pos][batch_pos:batch_pos+batch_size]
#     except:
#         return False,True,None,None,0,0
    batch_pos+=batch_size
    if len(now_datas)<batch_size:
        batch_pos=0
        is_eof = True
    # ===========================================================
    batch_size = len(now_datas) # 因為有可能在檔案的尾巴 讀不滿完整的batch_size 故在此更新batch_size
    # ==================若現在的檔案讀完============================
    if is_eof == True:
        random.shuffle(train_data_dict[len_pos])
        if len_pos == min_slen: # 一個iteration跑完 重新再跑
            len_pos = max_slen
            is_eoi = True
        else:                      # 現在這長度的file完了 開始讀len_pos-1的檔案
            while len_pos >= min_slen:
                len_pos -= 1
                if len_pos in train_data_dict:
                    break
            
            
        if batch_size == 0: # 剛好讀到尾巴了
            return is_eoi,is_eof,None,None,slen,batch_size
    # ============================================================
        
    score = np.array([i[0] for i in now_datas])
    cmmt = np.array([i[1]+[pad_ix]*(70-slen) for i in now_datas])
    return is_eoi,is_eof,score,cmmt,slen,batch_size

In [13]:
lr = 0.001                   
training_iters = 100      

n_vocab_size = vocab_size
sequence_length = 70   # 一句多少個字
embedding_size = 300
filter_sizes = [3,4,5]
num_filters = 100 
num_classes = 2
n_batch_size = 500
dropout_rate = 0.5

In [14]:
tf.reset_default_graph()

In [15]:
"""
    CNN 用來做情感分析
    Model structure : input layer -> convolution layer -> max-pooling -> softmax
"""

# input 的 placeholder , x [batch_size, sequence_length] -> 因為句長不固定所以第二維sequence length 設為placeholder
input_x = tf.placeholder(tf.int32, [None,None], name="input_x")
input_y = tf.placeholder(tf.int32, [None,], name="input_y")
ph_batch_size = tf.placeholder(tf.int32,[], name="ph_batch_size")
drop_rate = tf.placeholder(tf.float32,[], name="drop_rate")
"""
    設定word embedding , 使用 embedding lookup (給我字的id,回傳word_embedding)
    有人說tensorflow 目前不支援使用 gpu 操作embedding (?)
    tf.name_scope 會創造一個top-level 的 node,並將所有相關的 operation 加到 node 上
    在TensorBoard 可以看得比較清楚
"""
# with tf.name_scope("embedding"):

    # 預設word embedding 為 -1 ~ 1 之間的 亂數
w_Embd = tf.Variable(
        tf.random_uniform([vocab_size, embedding_size], -1.0 , 1.0),
        name = "w_Embd")

# embedding lookup 回傳 [batch_size, sequence_length, word_embedding]
embedded_words = tf.nn.embedding_lookup(w_Embd, input_x) 
y_embed = tf.constant(np.eye(num_classes))
y_emb = tf.nn.embedding_lookup(y_embed,input_y)
# tensorflow 的 conV2d 吃的是4維的向量,因此將其expand 成4維
embedded_words_expand = tf.expand_dims(embedded_words, -1)
embedded_words_expand = tf.transpose(embedded_words_expand,[0,1,3,2])

In [16]:
"""
    這邊根據不同的filter size 先各自創造自己的convolution layer + max-pooling layer
    最後再合併變成一個大的feature map
"""
max_pooled_output = []
for i, filter_size in enumerate(filter_sizes):

    # Convolution layer
    #filter_shape = [filter_size, embedding_size , 1 , num_filters]
    filter_shape = [filter_size, 1 , embedding_size , num_filters]

    # initialize weight and bias
    weight = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="weight")
    bias = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="bias")

    #     if(i > 0):
    #         tf.get_variable_scope().reuse_variables()

    conv = tf.nn.conv2d(
        embedded_words_expand,
        weight,
        strides=[1,1,1,1],
        padding="SAME",    #narrow conv
        name="conv"
    ) 

    # 過relu
    h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu") # =>(bsize,seq,1,fil)

    # max-pooling
    # ksize 這邊的 第二維 就是指filter 總共會滑過去幾次 -> sequence_length - filter_size + 1
    # input : [1,0,3,4] , filter_size : 2 , output: [[1,0],[0,3],[3,4]] -> length : 4 - 2 + 1 = 3 
    
    pooled = tf.nn.max_pool(
        h,
        ksize=[1, sequence_length , 1, 1],     
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool" 
    )

    max_pooled_output.append(pooled) # => [(bsize,1,1,fil)]*fil_size

# 合併 所有的 feature
num_filters_total = num_filters * len(filter_sizes)
h_pool = tf.concat(max_pooled_output, 3) # =>(bsize,1,1,fil*fil_size)
h_pool_flat = tf.reshape(h_pool, [-1,num_filters_total]) # =>(bsize,fil*fil_size)


In [17]:
# !!!!!!!!!!!
# dropout layer
# with tf.name_scope("dropout"):

h_drop = h_pool_flat
#h_drop = tf.nn.dropout(h_pool_flat, dropout_rate) # =>(bsize,fil*fil_size)

In [18]:
# Fully-connected Layer , calculate score , prediction
# with tf.name_scope("output"):
W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
scores = tf.nn.xw_plus_b(h_drop, W, b, name="scores") # =>(bsize,num_classes)
predictions = tf.argmax(scores, 1, name="predictions") # =>(bsize,1)

In [19]:
# Calculate mean cross-entropy loss
# with tf.name_scope("loss"):
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_emb, logits=scores))
# (bsize,num_classes),(bsize,num_classes)=>float

train_op = tf.train.AdamOptimizer(lr).minimize(loss)    

In [20]:
# Calculate Accuracy
# with tf.name_scope("accuracy"):
correct_predictions = tf.equal(predictions, tf.argmax(y_emb, 1)) # (bsize,1),(bsize,1)
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


In [17]:
# initialize 
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [37]:
# Train
saver = tf.train.Saver()
for i in range(training_iters):
    is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
    batch_now = 0
    while(not is_eoi):
        batch_now += 1
        if batch_size == 0:
            is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
            continue
        inp = {input_x:cmmt,input_y:score,ph_batch_size:batch_size,drop_rate:dropout_rate}
        _,c,a = sess.run([train_op,loss,accuracy],inp)
#         b = sess.run(pooled,inp)
#         c = sess.run(h_pool,inp)
#         input()
        if batch_now%200==0:
            print("iter:%d batch:%d loss:%f accuracy:%f %d"%(i,batch_now,c,a,slen))
            #     if (batch_now % 100 == 0):
        is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
    save_path = saver.save(sess,"./CNN1_model.bin")

iter:0 batch:200 loss:0.198101 accuracy:0.945055 55
iter:0 batch:400 loss:0.101613 accuracy:0.948454 52
iter:0 batch:600 loss:0.168690 accuracy:0.925234 47
iter:1 batch:200 loss:0.248023 accuracy:0.908046 58
iter:1 batch:400 loss:0.169457 accuracy:0.912088 55
iter:1 batch:600 loss:0.085744 accuracy:0.979381 52
iter:1 batch:800 loss:0.133199 accuracy:0.963303 46
iter:2 batch:200 loss:0.141411 accuracy:0.919540 58
iter:2 batch:400 loss:0.049070 accuracy:0.989011 55
iter:2 batch:600 loss:0.199310 accuracy:0.927835 52
iter:2 batch:800 loss:0.095824 accuracy:0.972477 46
iter:3 batch:200 loss:0.154549 accuracy:0.931035 58
iter:3 batch:400 loss:0.093724 accuracy:0.956044 55
iter:3 batch:600 loss:0.206170 accuracy:0.969072 52
iter:3 batch:800 loss:0.078451 accuracy:0.972477 46
iter:4 batch:200 loss:0.104470 accuracy:0.931035 58
iter:4 batch:400 loss:0.103287 accuracy:0.967033 55
iter:4 batch:600 loss:0.074814 accuracy:0.979381 52
iter:4 batch:800 loss:0.150094 accuracy:0.935780 46
iter:5 batch

In [19]:
b.shape

(72, 70, 300, 100)

In [21]:
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess,'%s/CNN1_model.bin'%hw_dir)

In [22]:
batch_pos

0

In [23]:
batch_size

1280

In [24]:
ans_list = []
is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
batch_now = 0
while(not is_eoi):
    batch_now += 1
    if batch_size == 0:
        is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
        continue
    inp = {input_x:cmmt,input_y:score,ph_batch_size:batch_size,drop_rate:dropout_rate}
    ans = sess.run(predictions,inp)
#     print("fuck")
    ans_list.append([cmmt,score,ans])
    is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)

In [25]:
ans_list2 = []
for ans in ans_list:
    bsize = ans[0].shape[0]
    ans[0] = ans[0].tolist()
    ans[1] = ans[1].tolist()
    ans[2] = ans[2].tolist()
    for i in range(bsize):
        cmmt = ''.join([ix2word[j] for j in ans[0][i]])
        ans_list2.append([cmmt,ans[1][i],ans[2][i]])

In [26]:
yes = 0
yes_no = 0
for i in ans_list2:
    if i[1]==i[2]:
        yes+=1
    yes_no +=1

In [27]:
yes/float(yes_no)

0.9394168466522678

In [28]:
different_CNN = [ans for ans in ans_list2]

In [29]:
with open('%s/output_diff_CNN.bin'%hw_dir,'wb') as f1:
    pickle.dump(different_CNN,f1)